In [1]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.metrics import mean_squared_error
import os
import time

In [63]:
train_data.columns

Index(['Unnamed: 0', 'stock', 'Timestamp', 'True_target', 'True_close',
       'close-slope-180', 'close-slope-90', 'close-slope-30', 'close-slope-7',
       'close-slope-3', 'close-r2-180', 'close-r2-90', 'close-r2-30',
       'close-r2-7', 'close-r2-3', 'vol-slope-180', 'vol-slope-90',
       'vol-slope-30', 'vol-slope-7', 'vol-slope-3', 'vol-r2-180', 'vol-r2-90',
       'vol-r2-30', 'vol-r2-7', 'vol-r2-3', 'close_today', 'close_tomorrow'],
      dtype='object')

In [62]:
### Unpack data

data_dir = 'main/data_final/'

train_data = pd.read_csv(f'{data_dir}/train_data.csv')
test_data = pd.read_csv(f'{data_dir}/test_data.csv')

# X_features = ['diff_close_180', 'diff_close_90',
#        'diff_close_30', 'diff_close_7', 'diff_close_3', 'std_close_180',
#        'std_close_90', 'std_close_30', 'std_close_7', 'std_close_3',
#        'avg_vol_180', 'avg_vol_90', 'avg_vol_30', 'avg_vol_7', 'avg_vol_3']


# train_X = train_data[X_features]
# train_y = train_data[['target']]
# test_X = test_data[X_features]
# test_y = test_data[['target']]

# print(f'train_X: {train_X.shape}')
# print(f'train_y: {train_y.shape}')
# print(f'test_X: {test_X.shape}')
# print(f'test_y: {test_y.shape}')


In [57]:
params_xgd = {
		'max_depth': 7,
		'objective': 'reg:squarederror',
		'learning_rate': 0.1,
		'n_estimators': 1000,
		'eval_metric': 'rmse',
		'early_stopping_rounds': 20
	}
model = xgb.XGBRegressor(**params_xgd)

In [58]:
model.fit(train_X, train_y, eval_set = [(test_X, test_y)])


[0]	validation_0-rmse:1.76472
[1]	validation_0-rmse:1.75615
[2]	validation_0-rmse:1.74896
[3]	validation_0-rmse:1.74377
[4]	validation_0-rmse:1.73834
[5]	validation_0-rmse:1.73434
[6]	validation_0-rmse:1.73093
[7]	validation_0-rmse:1.72858
[8]	validation_0-rmse:1.72615
[9]	validation_0-rmse:1.72427
[10]	validation_0-rmse:1.72278
[11]	validation_0-rmse:1.72184
[12]	validation_0-rmse:1.72097
[13]	validation_0-rmse:1.72007
[14]	validation_0-rmse:1.71930
[15]	validation_0-rmse:1.71862
[16]	validation_0-rmse:1.71817
[17]	validation_0-rmse:1.71786
[18]	validation_0-rmse:1.71738
[19]	validation_0-rmse:1.71746
[20]	validation_0-rmse:1.71734
[21]	validation_0-rmse:1.71708
[22]	validation_0-rmse:1.71691
[23]	validation_0-rmse:1.71689
[24]	validation_0-rmse:1.71680
[25]	validation_0-rmse:1.71685
[26]	validation_0-rmse:1.71676
[27]	validation_0-rmse:1.71678
[28]	validation_0-rmse:1.71687
[29]	validation_0-rmse:1.71700
[30]	validation_0-rmse:1.71688
[31]	validation_0-rmse:1.71700
[32]	validation_0-

XGBRegressor(base_score=0.5, booster='gbtree', callbacks=None,
             colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
             early_stopping_rounds=20, enable_categorical=False,
             eval_metric='rmse', feature_types=None, gamma=0, gpu_id=-1,
             grow_policy='depthwise', importance_type=None,
             interaction_constraints='', learning_rate=0.1, max_bin=256,
             max_cat_threshold=64, max_cat_to_onehot=4, max_delta_step=0,
             max_depth=7, max_leaves=0, min_child_weight=1, missing=nan,
             monotone_constraints='()', n_estimators=1000, n_jobs=0,
             num_parallel_tree=1, predictor='auto', random_state=0, ...)

In [59]:
predict_y = model.predict(test_X)

In [60]:
mean_squared_error(test_y, predict_y)


2.9472545430512844

In [61]:
primary_key = test_data[['stock', 'Timestamp']]
diff = abs((test_y.values.T[0] - predict_y)/test_y.values.T[0])*100
abs_close = test_data[['True_close']].values.T[0]
abs_target = test_data[['True_target']].values.T[0]
abs_prediction = abs_close*predict_y/100+abs_close
abs_diff = abs_target-abs_prediction
# diff = predict_y
output = pd.DataFrame(np.dstack((test_data.stock, test_data.Timestamp, test_y.values.T[0], predict_y, diff, abs_close, abs_target, abs_prediction, abs_diff))[0], columns=['TICK', 'Timestamp', 'true', 'predict', '% diff', 'abs_close', 'abs_true', 'abs_predict', 'abs_diff'])

output.to_csv('main/results/output_1.csv')
output

C:\Users\calvi\AppData\Local\Temp\ipykernel_19820\1932781497.py:2: RuntimeWarning: divide by zero encountered in divide
  diff = abs((test_y.values.T[0] - predict_y)/test_y.values.T[0])*100


,TICK,Timestamp,true,predict,% diff,abs_close,abs_true,abs_predict,abs_diff
0,0700.HK,2018-09-24,0.657885,0.086817,86.803592,325.992371,328.180206,326.275388,1.904818
1,0700.HK,2018-09-26,-1.403289,0.109273,107.786923,328.180206,323.605591,328.538819,-4.933228
2,0700.HK,2018-09-27,-0.666657,0.094237,114.13576,323.605591,321.417755,323.910547,-2.492792
3,0700.HK,2018-09-28,-1.905354,0.068291,103.584175,321.417755,315.251923,321.637255,-6.385333
4,0700.HK,2018-10-02,-0.309413,0.068289,122.070393,315.251923,314.257416,315.467204,-1.209788
...,...,...,...,...,...,...,...,...,...
9603,XOM,2019-12-23,-0.386047,0.058213,115.079148,66.662033,66.405968,66.700839,-0.294871
9604,XOM,2019-12-24,0.156497,0.087853,43.862933,66.405968,66.510292,66.464307,0.045985
9605,XOM,2019-12-26,-0.342763,0.029244,108.531935,66.510292,66.282677,66.529743,-0.247066
9606,XOM,2019-12-27,-0.584619,0.078756,113.471246,66.282677,65.893845,66.334878,-0.441033
